# Introduction

The COMPAS simulations might be very large in data size
while the actual data you need to reproduce your results could
be small. Hence it might make sense to only publish a reduced number
of files and columns based on some criteria.

Here we show how you can reduce your data.
The main things you need are:
    
1 - the seeds you want to have in your data

2 - the files you want in your data

3 - the columns you want for each file

The plain python script to do this is placed in the H5 folder 
in postProcessing. Here we just show an example of how to reduce the data and call
the script



# Paths needed

In [1]:
pathToData = '/home/cneijssel/Desktop/Test/COMPAS_output.h5'

# Imports

In [2]:
import h5py  as h5   #for handling data format
import numpy as np  #for array handling
import os           #For checking existence data
import sys

pathNoteBook     = os.getcwd()
pathPythonScript = pathNoteBook + '/PythonScripts/'
#custom scripts
sys.path.append(pathPythonScript)
import rewrite_H5

# 1  Loading Data

In [3]:
Data  = h5.File(pathToData)
print("The main files I have at my disposal are:\n",list(Data.keys()))
#Say you want only the double neutron stars which
#But you do want masses of all the initial systems (i.e. different seeds per file)
#but for some reason want systems in which one or more systems had a USSN 
#and you dont want the CommonEnvelopes file


The main files I have at my disposal are:
 ['CommonEnvelopes', 'DoubleCompactObjects', 'RLOF', 'Supernovae', 'SystemParameters']


# 2 Files/Columns per file do I want?

We use dictionaries to specifically link all the entries

In [4]:


# Which Files do I want?
filesOfInterest   = {1:'DoubleCompactObjects',\
                     2:'Supernovae',\
                     3:'SystemParameters'}

#Give a list of columns you want, if you want all, say ['All']
columnsOfInterest = {1:['All'],\
                     2:['All'],\
                     3:['SEED', 'Mass@ZAMS_1', 'Mass@ZAMS_2']}

#example of the seeds dictionary the actual one will be defined later
seedsOfInterest   = {1:None,\
                     2:None,\
                     3:None}

# 3 Which seeds do I want per file?

In [5]:
##### I want all the systems

seedsSystems = Data['SystemParameters']['SEED'][()]



#### Of the double COMPCt objects only the DNSs

DCOs = Data['DoubleCompactObjects']

typePrimary   =  DCOs['Stellar_Type_1'][()]
typeSecondary =  DCOs['Stellar_Type_1'][()]
seeds         =  DCOs['SEED'][()]

DNSs          =  (typePrimary == 13) & (typeSecondary == 13)
seedsDNS      =  seeds[DNSs]



######## Only disrupted SN 

SNe  = Data['Supernovae']

seeds     = SNe['SEED']
USSN      = SNe['Unbound'][()] == True
seedsUSSN = seeds[USSN]



####put in dictionary (DOUBLE CHECK ORDER :) )

seedsOfInterest   = {1:seedsDNS,\
                     2:seedsUSSN,\
                     3:seedsSystems}


# 4 Call the function which creates the h5 file

In [6]:
pathToNewData = '/home/cneijssel/Desktop/Test/COMPAS_outputNew.h5'

rewrite_H5.reduceH5(pathToOld = pathToData, pathToNew = pathToNewData,\
                     dictFiles=filesOfInterest, dictColumns=columnsOfInterest, dictSeeds=seedsOfInterest)

In [7]:
rewrite_H5.printAllColumnsInH5(pathToNewData)

Filename = DoubleCompactObjects
----------------------
	   column name                             unit                length
	   --------------------------------------------------------------------
	   Coalescence_Time                        Myr                 107
	   Eccentricity@DCO                                            107
	   ID                                                          107
	   MT_Case_1                                                   107
	   MT_Case_2                                                   107
	   --------------------------------------------------------------------
	   Mass_1                                  Msol                107
	   Mass_2                                  Msol                107
	   Merges_Hubble_Time                      State               107
	   Recycled_NS_1                           Event               107
	   Recycled_NS_2                           Event               107
	   --------------------------------------------